<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Chat_with_MultiplePDFs_Mistral_7B_Instruct1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=36770cd83b183c2f5e8d5c0240b6d4a462a5dc9edfdb553

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
#load pdf files
loader = PyPDFDirectoryLoader("/content/sample_data/Data/")
data = loader.load()

In [ ]:
print(data)

[Document(page_content='/two.ltab/two.ltab MAY /two.lprp/zero.lprp/one.lprp/nine.lprp  /  HOMETEXTILESTODAY.COMIn Walmart Inc.’s new annual report, Doug McMillon reﬂ  ects on lessons \nlearned during his ﬁ  ve years as the company’s chief executive ofﬁ  cer.\nThe report, titled “Deﬁ  ning the Future of Retail,” was recently posted to \nthe company’s investor relations site ahead of the retailer’s June 5 annual \nshareholder meeting\nMcMillon writes that Walmart is focused on long-term success.\n“We’re playing the long game,” he said. “Managing our business on a dai-\nly basis is important, but our most important strategic decisions are made \nin light of what we want our company \nto become for the next generation.”\nA leader during times of transforma-\ntion has to be curious, introduce new \nideas and ask questions. “You can’t \npush a rope, but you can pull it. In \nother words, sometimes you just can’t \nlead from behind,” he added.\nIn pursuing growth, companies also \nhave to get

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

23

In [ ]:
#get the third chunk
text_chunks[2]

Document(page_content='Econometric Computing with HC and HAC Covariance\nMatrix Estimators\nAchim Zeileis\nUniversit ¨at Innsbruck\nAbstract\nThis introduction to the Rpackagesandwich is a (slightly) modiﬁed version of Zeileis\n(2004), published in the Journal of Statistical Software . A follow-up paper on object\nobject-oriented computation of sandwich estimators is available in ( Zeileis 2006b ).\nData described by econometric models typically contains autocorrel ation and/or het-\neroskedasticity of unknown form and for inference in such models it is essential to use\ncovariance matrix estimators that can consistently estimate the covar iance of the model\nparameters. Hence, suitable heteroskedasticity-consistent (HC) an d heteroskedasticity\nand autocorrelation consistent (HAC) estimators have been receiving attention in the\neconometric literature over the last 20 years. To apply these estimat ors in practice,\nan implementation is needed that preferably translates the conceptu al

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
#connect to google drive
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/Model/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [ ]:
query = "What is linear regression model"

In [ ]:
qa.run(query)

' The linear regression model is a statistical method for modeling the relationship between a dependent variable and one or more independent variables, commonly referred to as predictors or explanatory variables. In this paper, the linear regression model is used as a starting point to derive estimators for the variance-covariance matrix of the errors in certain structural change models.'

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Llama.generate: prefix-match hit


Answer:  It is a statistical method used to analyze the linear relationship between two or more variables. In this case, it is used to estimate the coefficients of a linear regression model.


Llama.generate: prefix-match hit


Answer:  sandwich provides various functions for estimating the covariance matrix. The most commonly used ones are vcovHAC and vcovHC. These functions can take different weighting schemes, including kernel-based HAC estimation with automatic bandwidth selection based on weightsAndrews andbwAndrews. Other functions like weightsLumley provide different weighting schemes, such as truncated and smoothed weights, which are useful in certain applications. In econometric analyses, these functions are used to compute partial t-tests for assessing the significance of a parameter. The choice of estimator depends on the presence of heteroscedasticity and autocorrelation in the errors.
